In [41]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import databento as db

In [43]:
def db_preparing(path: str) -> pd.DataFrame:
    store = db.DBNStore.from_file(path)
    df = store.to_df()
    df = df.reset_index(drop=True)
    trades = df[['ts_event', 'side', 'depth', 'price', 'size']]
    trades['ts_event'] = pd.to_datetime(trades['ts_event'].astype('int64'))
    trades = trades.set_index('ts_event')
    return trades

In [45]:
def resampling(path: str) -> pd.DataFrame:
    trades = db_preparing(path)
    sides = ['A', 'B', 'N']
    res_trds = []
    for side in sides:
        side_trades = trades[trades['side'] == f'{side}']
        resampled_trades = side_trades.resample("100ms").agg({"price": "mean", "size": "sum", "depth": "max"}).fillna(0)
        res_trds.append(resampled_trades['price'])
        res_trds.append(resampled_trades['size'])
    resampled_depth = trades['depth'].resample("100ms").max().fillna(0)
    res_trds.append(resampled_depth)
    trades_resampled = pd.DataFrame(res_trds).T
    trades_resampled = trades_resampled.fillna(0)
    trades_resampled.columns = ['trade_price_A', 'trade_volume_A', 'trade_price_B', 'trade_volume_B', 'trade_price_N', 'trade_volume_N', 'depth']
    return trades_resampled

In [ ]:
# path_trades = r'C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250701.trades.FAST.dbn.zst'
# trades_resampled = resampling(path_trades)  

In [ ]:
path_trades = r'C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250701.trades.FAST.dbn.zst'
trades_resampled = resampling(path_trades) 

In [51]:
path_trades_ANSS = r"C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250625.trades.ANSS.dbn.zst"
ANSS_trades_resampled = resampling(path_trades_ANSS)

path_trades_FAST = r"C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250625.trades.FAST.dbn.zst"
FAST_trades_resampled = resampling(path_trades_FAST)

path_trades_SPY = r"C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250625.trades.SPY.dbn.zst"
SPY_trades_resampled = resampling(path_trades_SPY)

path_trades_VXX = r"C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250625.trades.VXX.dbn.zst"
VXX_trades_resampled = resampling(path_trades_VXX)

path_trades_VRSK = r"C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250625.trades.VRSK.dbn.zst"
VRSK_trades_resampled = resampling(path_trades_VRSK)

path_trades_QQQ = r"C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250625.trades.QQQ.dbn.zst"
QQQ_trades_resampled = resampling(path_trades_QQQ)

path_trades_MRVL = r"C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250625.trades.MRVL.dbn.zst"
MRVL_trades_resampled = resampling(path_trades_MRVL)

In [53]:
qqq_table = pa.Table.from_pandas(QQQ_trades_resampled)
pq.write_table(qqq_table, 'QQQ_0625.parquet')

spy_table = pa.Table.from_pandas(SPY_trades_resampled)
pq.write_table(spy_table, 'SPY_0625.parquet')

mrvl_table = pa.Table.from_pandas(MRVL_trades_resampled)
pq.write_table(mrvl_table, 'MRVL_0625.parquet')

vxx_table = pa.Table.from_pandas(VXX_trades_resampled)
pq.write_table(vxx_table, 'VXX_0625.parquet')

vrsk_table = pa.Table.from_pandas(VRSK_trades_resampled )
pq.write_table(vrsk_table, 'VRSK_0625.parquet')

fast_table = pa.Table.from_pandas(FAST_trades_resampled)
pq.write_table(fast_table, 'FAST_0625.parquet')

anss_table = pa.Table.from_pandas(ANSS_trades_resampled)
pq.write_table(anss_table, 'ANSS_0625.parquet')